In [81]:
import numpy as np
import pandas as pd
import pymap3d
from numpy.linalg import inv
import warnings
from gnatss.loaders import read_novatel_L1_data_files
from gnatss.loaders import load_gps_solutions

warnings.filterwarnings('ignore')

In [82]:
inspvaa_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSPVAA.dat'], data_format="INSPVAA")

In [83]:
insstdeva_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSSTDEVA.dat'], data_format="INSSTDEVA")
 

In [84]:
inspvaa_df = inspvaa_df.rename(
    columns={
        "ant_e": "vel_e",
        "ant_n": "vel_n",
        "ant_u": "vel_u",
        "time":"dts",
    },
    errors="raise"
)
inspvaa_df = inspvaa_df[
    [
        'dts', 'vel_e', 'vel_n', 'vel_u',
    ]
]

In [85]:
insstdeva_df = insstdeva_df.rename(
    columns={
        "ant_e std": "v_sde",
        "ant_n std": "v_sdn",
        "ant_u std": "v_sdu",
        "time":"dts",
    },
    errors="raise"
)
insstdeva_df = insstdeva_df[
    [
        'dts',
        'v_sde', 'v_sdn', 'v_sdu'
    ]
]

insstdeva_df["v_sden"] = 0.0
insstdeva_df["v_sdeu"] = 0.0
insstdeva_df["v_sdnu"] = 0.0


In [86]:
inspvaa_df

,dts,vel_e,vel_n,vel_u
0,7.122935e+08,0.1014,-0.6858,0.2413
1,7.122935e+08,0.0788,-0.6380,0.2734
2,7.122935e+08,0.0534,-0.5833,0.2926
3,7.122935e+08,0.0405,-0.5440,0.2950
4,7.122935e+08,0.0346,-0.5122,0.2801
...,...,...,...,...
3865082,7.123618e+08,1.0631,-0.9726,-0.1158
3865083,7.123618e+08,1.0885,-0.9804,-0.1416
3865084,7.123618e+08,1.1255,-0.9709,-0.1728
3865085,7.123618e+08,1.1578,-0.9542,-0.2087


In [87]:
insstdeva_df

,dts,v_sde,v_sdn,v_sdu,v_sden,v_sdeu,v_sdnu
0,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
1,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
2,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
3,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
4,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3865303,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865304,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865305,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865306,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0


In [88]:
"""
def load_gps_solutions(
    files: List[str], time_round: int = constants.DELAY_TIME_PRECISION
) -> pd.DataFrame:
"""
gps_df = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/209/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/210/GPS_PPP/GPS_POS_FREED',

    ],
    columns=["time", "dtype", "x", "y", "z", "sdx", "sdy", "sdz"],
)
gps_df = gps_df.rename(
    columns={
        "time":"dts",
    },
    errors="raise"
)

gps_df = gps_df[
    [
        'dts', 'x', 'y', 'z', 'sdx', 'sdy', 'sdz'
    ]
]

gps_df["sdxy"] = np.sqrt(gps_df.sdx * gps_df.sdy)
gps_df["sdxz"] = np.sqrt(gps_df.sdx * gps_df.sdz)
gps_df["sdyz"] = np.sqrt(gps_df.sdy * gps_df.sdz)

gps_df

,dts,x,y,z,sdx,sdy,sdz,sdxy,sdxz,sdyz
0,712207183.0,-2.574893e+06,-3.681574e+06,4.512097e+06,2.4491,2.4489,2.4488,2.449000,2.448950,2.448850
1,712207184.0,-2.574893e+06,-3.681574e+06,4.512097e+06,2.2359,2.2357,2.2356,2.235800,2.235750,2.235650
2,712207185.0,-2.574893e+06,-3.681574e+06,4.512097e+06,2.0000,1.9998,1.9997,1.999900,1.999850,1.999750
3,712207186.0,-2.574893e+06,-3.681574e+06,4.512097e+06,1.7323,1.7320,1.7319,1.732150,1.732100,1.731950
4,712207187.0,-2.574893e+06,-3.681574e+06,4.512097e+06,1.4147,1.4144,1.4142,1.414550,1.414450,1.414300
...,...,...,...,...,...,...,...,...,...,...
193288,712400473.0,-2.575283e+06,-3.682610e+06,4.511038e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713
193289,712400474.0,-2.575284e+06,-3.682610e+06,4.511038e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713
193290,712400475.0,-2.575284e+06,-3.682610e+06,4.511038e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713
193291,712400476.0,-2.575284e+06,-3.682609e+06,4.511037e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713


In [89]:
merged_df = inspvaa_df.merge(gps_df, on="dts", how="left")
merged_df = merged_df.merge(insstdeva_df, on="dts", how="left")
merged_df.reset_index(drop=True)
merged_df.sort_values("dts").reset_index(drop=True)

,dts,vel_e,vel_n,vel_u,x,y,z,sdx,sdy,sdz,sdxy,sdxz,sdyz,v_sde,v_sdn,v_sdu,v_sden,v_sdeu,v_sdnu
0,7.122072e+08,0.1631,-1.4049,0.1295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
1,7.122072e+08,0.1743,-1.4157,0.0827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
2,7.122072e+08,0.1818,-1.4301,0.0490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
3,7.122072e+08,0.1824,-1.4465,0.0160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
4,7.122072e+08,0.1820,-1.4636,-0.0063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865082,7.124005e+08,-1.3296,0.0802,0.3085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0
3865083,7.124005e+08,-1.3541,0.0458,0.2389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0
3865084,7.124005e+08,-1.3617,0.0044,0.1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0
3865085,7.124005e+08,-1.3508,-0.0325,0.0253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0


In [90]:
merged_df.iloc[0:50]

,dts,vel_e,vel_n,vel_u,x,y,z,sdx,sdy,sdz,sdxy,sdxz,sdyz,v_sde,v_sdn,v_sdu,v_sden,v_sdeu,v_sdnu
0,7.122935e+08,0.1014,-0.6858,0.2413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0159,0.0179,0.0160,0.0,0.0,0.0
1,7.122935e+08,0.0788,-0.6380,0.2734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0159,0.0179,0.0160,0.0,0.0,0.0
2,7.122935e+08,0.0534,-0.5833,0.2926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0159,0.0179,0.0160,0.0,0.0,0.0
3,7.122935e+08,0.0405,-0.5440,0.2950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0159,0.0179,0.0160,0.0,0.0,0.0
4,7.122935e+08,0.0346,-0.5122,0.2801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0159,0.0179,0.0160,0.0,0.0,0.0
5,7.122935e+08,0.0341,-0.4850,0.2472,-2.575324e+06,-3.682599e+06,4.511020e+06,0.0119,0.0143,0.0124,0.013045,0.012147,0.013316,0.0159,0.0179,0.0160,0.0,0.0,0.0
6,7.122935e+08,0.0438,-0.4740,0.2121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0
7,7.122935e+08,0.0576,-0.4702,0.1735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0
8,7.122935e+08,0.0727,-0.4730,0.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0
9,7.122935e+08,0.0881,-0.4780,0.1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0


In [91]:
first_pos = merged_df[~merged_df.x.isnull()].iloc[0].name
merged_df = merged_df.loc[first_pos:].reset_index(drop=True)
df = merged_df.to_records()

In [92]:
merged_df

,dts,vel_e,vel_n,vel_u,x,y,z,sdx,sdy,sdz,sdxy,sdxz,sdyz,v_sde,v_sdn,v_sdu,v_sden,v_sdeu,v_sdnu
0,7.122935e+08,0.0341,-0.4850,0.2472,-2575324.461,-3.682599e+06,4.511020e+06,0.0119,0.0143,0.0124,0.013045,0.012147,0.013316,0.0159,0.0179,0.0160,0.0,0.0,0.0
1,7.122935e+08,0.0438,-0.4740,0.2121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0
2,7.122935e+08,0.0576,-0.4702,0.1735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0
3,7.122935e+08,0.0727,-0.4730,0.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0
4,7.122935e+08,0.0881,-0.4780,0.1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160,0.0181,0.0165,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865077,7.123618e+08,1.0631,-0.9726,-0.1158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865078,7.123618e+08,1.0885,-0.9804,-0.1416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865079,7.123618e+08,1.1255,-0.9709,-0.1728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865080,7.123618e+08,1.1578,-0.9542,-0.2087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0147,0.0172,0.0206,0.0,0.0,0.0


In [93]:
import gc

del [[inspvaa_df,insstdeva_df, gps_df, ]]
gc.collect()

0

In [94]:
from math import pi

import numpy as np
from numpy import asarray, empty_like, finfo, where
from numpy import arctan as atan
from numpy import arctan2 as atan2
from numpy import (
    cos,
    degrees,
    hypot,
    sin,
    sqrt,
    tan,
)

import numba

In [95]:
# Process noise matrix
gnss_pos_psd = 3.125e-5
vel_psd = 0.0025

WGS84_ELL = {"name": "WGS-84 (1984)", "a": 6378137.0, "b": 6356752.31424518}
semimajor_axis = WGS84_ELL["a"]
semiminor_axis = WGS84_ELL["b"]


@numba.njit
def ecef2geodetic(
    x: float,
    y: float,
    z: float,
    semimajor_axis: float = semimajor_axis,
    semiminor_axis: float = semiminor_axis,
    eps=finfo(np.float32).eps,
) -> tuple:
    """
    convert ECEF (meters) to geodetic coordinates

    rewrite of pymap3d.ecef2geodetic to use numba for speedup...
    not as robust
    """

    x = asarray(x)
    y = asarray(y)
    z = asarray(z)

    r = sqrt(x**2 + y**2 + z**2)

    E = sqrt(semimajor_axis**2 - semiminor_axis**2)

    # eqn. 4a
    u = sqrt(0.5 * (r**2 - E**2) + 0.5 * hypot(r**2 - E**2, 2 * E * z))

    hxy = hypot(x, y)

    huE = hypot(u, E)

    # eqn. 4b
    Beta = empty_like(r)
    Beta = atan(huE / u * z / hxy)
    # eqn. 13
    Beta += ((semiminor_axis * u - semimajor_axis * huE + E**2) * sin(Beta)) / (
        semimajor_axis * huE * 1 / cos(Beta) - E**2 * cos(Beta)
    )

    # eqn. 4c
    # %% final output
    lat = atan(semimajor_axis / semiminor_axis * tan(Beta))

    # # patch latitude for float32 precision loss
    lim_pi2 = pi / 2 - eps
    lat = where(Beta >= lim_pi2, pi / 2, lat)
    lat = where(Beta <= -lim_pi2, -pi / 2, lat)

    lon = atan2(y, x)

    # eqn. 7
    cosBeta = cos(Beta)

    # patch altitude for float32 precision loss
    cosBeta = where(Beta >= lim_pi2, 0, cosBeta)
    cosBeta = where(Beta <= -lim_pi2, 0, cosBeta)

    alt = hypot(z - semiminor_axis * sin(Beta), hxy - semimajor_axis * cosBeta)

    lat = degrees(lat)
    lon = degrees(lon)

    return lat, lon, alt


@numba.njit
def predict(dt, X, P, Q, F):
    F[0:3, 3:6] = np.identity(3) * dt
    Q = updateQ(dt, Q)
    X = F @ X
    P = F @ P @ F.T + Q

    return X, P, Q, F


@numba.njit
def updateQ(dt, Q):
    # Position estimation noise
    # Initial Q values from Chadwell code 3.125d-5 3.125d-5 3.125d-5 0.0025 0.0025 0.0025, assumes white noise of 2.5 cm over a second
    Q[0:3, 0:3] = np.identity(3) * gnss_pos_psd * dt

    # Velocity estimation noise (acc psd)
    Q[3:6, 3:6] = np.identity(3) * vel_psd

    return Q


@numba.njit
def rot_vel(row, lat, lon):
    """
    ------------------- Rotate ENU velocity into ECEF velocity --------------------------------
                dX = |  -sg   -sa*cg  ca*cg | | de |        de = |  -sg       cg      0 | | dX |
                dY = |   cg   -sa*sg  ca*sg | | dn |  and   dn = |-sa*cg  -sa*sg     ca | | dY |
                dZ = |    0    ca     sa    | | du |        du = | ca*cg   ca*sg     sa | | dZ |
    -------------------------------------------------------------------------------------------
    """
    v_enu = row[1:4].copy().reshape((3, 1))
    lat = np.deg2rad(lat)
    lam = np.deg2rad(lon)

    ca = np.cos(lat)
    sa = np.sin(lat)

    cg = np.cos(lam)
    sg = np.sin(lam)

    rot = np.zeros((3, 3))
    rot[0, 0] = -sg
    rot[0, 1] = -sa * cg
    rot[0, 2] = ca * cg
    rot[1, 0] = cg
    rot[1, 1] = -sa * sg
    rot[1, 2] = ca * sg
    rot[2, 0] = 0
    rot[2, 1] = ca
    rot[2, 2] = sa
    v_xyz = rot @ v_enu

    return rot, v_xyz


@numba.njit
def update_vel_cov(row, R_velocity, rot):
    # Velocity measurement noise
    # Vel STD
    R_velocity[0, 0] = row[13] ** 2
    R_velocity[1, 1] = row[14] ** 2
    R_velocity[2, 2] = row[15] ** 2

    R_velocity[0, 1] = row[16]
    R_velocity[0, 2] = row[17]
    R_velocity[1, 2] = row[18]

    R_velocity[1, 0] = R_velocity[0, 1]
    R_velocity[2, 0] = R_velocity[0, 2]
    R_velocity[2, 1] = R_velocity[1, 2]

    R_velocity = rot @ R_velocity @ rot.T
    return R_velocity


@numba.njit
def update_rpos(row, R_position):
    R_position[0, 0] = row[7] ** 2
    R_position[1, 1] = row[8] ** 2
    R_position[2, 2] = row[9] ** 2

    R_position[0, 1] = np.sign(row[10]) * row[10] ** 2
    R_position[0, 2] = np.sign(row[11]) * row[11] ** 2
    R_position[1, 2] = np.sign(row[12]) * row[12] ** 2

    R_position[1, 0] = R_position[0, 1]
    R_position[2, 0] = R_position[0, 2]
    R_position[2, 1] = R_position[1, 2]
    return R_position


@numba.njit
def update_position(row, Nx, X, P, R_position):
    pos_xyz = row[4:7].copy().reshape((3, 1))

    H = np.zeros((3, Nx))
    H[0:3, 0:3] = np.identity(3)
    R_position = update_rpos(row, R_position)

    y = pos_xyz - H @ X
    S = H @ P @ H.T + R_position
    K = (P @ H.T) @ np.linalg.inv(S)
    X = X + K @ y

    I = np.identity(Nx)  # noqa
    P = (I - K @ H) @ P
    return X, P, R_position


@numba.njit
def update_velocity(Nx, X, P, R_velocity, v_xyz):
    H = np.zeros((3, Nx))
    H[0:3, 3:6] = np.identity(3)

    y = v_xyz - H @ X
    S = H @ P @ H.T + R_velocity
    K = (P @ H.T) @ np.linalg.inv(S)
    X = X + K @ y

    IDENTITY = np.identity(Nx)
    P = (IDENTITY - K @ H) @ P
    return X, P, R_velocity


@numba.njit
def rts_smoother(Xs, Ps, F, Q):
    # Rauch, Tongue, and Striebel smoother

    n, dim_x, _ = Xs.shape

    # smoother gain
    K = np.zeros((n, dim_x, dim_x))
    x, P, Pp = Xs.copy(), Ps.copy(), Ps.copy()

    i = 0
    for k in range(n - 2, -1, -1):
        Pp[k] = F @ P[k] @ F.T + Q  # predicted covariance

        K[k] = P[k] @ F.T @ np.linalg.inv(Pp[k])
        x[k] += K[k] @ (x[k + 1] - (F @ x[k]))
        P[k] += K[k] @ (P[k + 1] - Pp[k]) @ K[k].T
        i += 1

    return x, P, K, Pp


@numba.njit
def kalman_init(row):
    dt = 5e-2
    Nx = 6
    Nu = 3  # noqa

    # error states: pos_xyz, v_ned, eul, bias_acc, bias_gyro
    X = np.zeros((Nx, 1))
    # Initial antenna location
    pos_xyz = row[4:7].copy().reshape((3, 1))
    X[0:3] = pos_xyz

    lat, lon, _ = ecef2geodetic(X[0], X[1], X[2])
    # Initial antenna velocity
    rot, v_xyz = rot_vel(row, lat[0], lon[0])
    X[3:6] = v_xyz

    # Process model

    # State transition matrix
    F = np.identity(Nx)
    F[0:3, 3:6] = np.identity(3) * dt

    # Set initial values to 0.25?
    P = np.identity(Nx)

    # Process noise matrix
    Q = np.zeros((Nx, Nx))
    Q = updateQ(dt, Q)

    # Position measurement noise
    R_position = np.identity(3)
    R_position = update_rpos(row, R_position)

    # Velocity measurement noise
    R_velocity = np.identity(3) * 4e-8
    R_velocity[0, 1] = 1.5e-9
    R_velocity[0, 2] = 1.5e-9
    R_velocity[1, 2] = 1.5e-9
    R_velocity[1, 0] = R_velocity[0, 1]
    R_velocity[2, 0] = R_velocity[0, 2]
    R_velocity[2, 1] = R_velocity[1, 2]

    R_velocity = update_vel_cov(row, R_velocity, rot)

    return Nx, X, P, Q, F, R_position, R_velocity


@numba.njit
def run_filter_simulation(records):
    last_time = np.nan
    records_len = records.shape[0]
    Xs = np.zeros((records_len, 6, 1))
    Ps = np.zeros((records_len, 6, 6))
    n_records = len(records)
    for i in range(n_records):
        row = records[i]
        dts = row[0]
        if i == 0:
            Nx, X, P, Q, F, R_position, R_velocity = kalman_init(row)
            last_time = dts
        else:
            dt = np.abs(
                dts - last_time
            )  # This helps to stabilize the solution, abs ensures reverse filtering works.
            X, P, Q, F = predict(dt, X, P, Q, F)

            last_time = dts

            lat, lon, _ = ecef2geodetic(X[0], X[1], X[2])
            rot, v_xyz = rot_vel(row, lat[0], lon[0])

            # New velocity standard deviations
            if ~np.isnan(row[16]):
                R_velocity = update_vel_cov(row, R_velocity, rot)

            # new GNSS measurement
            if ~np.isnan(row[4]):
                X, P, R_position = update_position(row, Nx, X, P, R_position)

            # new velocity measurement
            if ~np.isnan(row[1]):
                X, P, R_velocity = update_velocity(Nx, X, P, R_velocity, v_xyz)

        Xs[i] = X
        Ps[i] = P

    x, P, K, Pp = rts_smoother(Xs, Ps, F, Q)

    return x, P, K, Pp

In [96]:
records = merged_df.to_numpy()
records

array([[ 7.12293533e+08,  3.41000000e-02, -4.85000000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12293533e+08,  4.38000000e-02, -4.74000000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12293533e+08,  5.76000000e-02, -4.70200000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 7.12361848e+08,  1.12550000e+00, -9.70900000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12361848e+08,  1.15780000e+00, -9.54200000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12361848e+08,  1.19180000e+00, -9.27200000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [97]:
%%time
x, P, K, Pp = run_filter_simulation(records)

CPU times: user 39 s, sys: 1.33 s, total: 40.3 s
Wall time: 40.1 s


In [98]:
records_copy = np.concatenate((records, np.zeros((records.shape[0], 3))), axis=1)

In [99]:
records_copy

array([[ 7.12293533e+08,  3.41000000e-02, -4.85000000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12293533e+08,  4.38000000e-02, -4.74000000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12293533e+08,  5.76000000e-02, -4.70200000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 7.12361848e+08,  1.12550000e+00, -9.70900000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12361848e+08,  1.15780000e+00, -9.54200000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.12361848e+08,  1.19180000e+00, -9.27200000e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [100]:
@numba.njit
def ecef2geodetic_v1(records):
    for i in range(len(records)):
        records[i, 19:22] = ecef2geodetic(records[i, 4],records[i, 5],records[i, 6])
    return records

In [101]:
%%time

records_copy_v1 = ecef2geodetic_v1(records_copy)
records_copy_v1

CPU times: user 3.65 s, sys: 17.2 ms, total: 3.67 s
Wall time: 3.66 s


array([[ 7.12293533e+08,  3.41000000e-02, -4.85000000e-01, ...,
         4.53022793e+01, -1.24965962e+02,  2.74717960e+01],
       [ 7.12293533e+08,  4.38000000e-02, -4.74000000e-01, ...,
                    nan,             nan,             nan],
       [ 7.12293533e+08,  5.76000000e-02, -4.70200000e-01, ...,
                    nan,             nan,             nan],
       ...,
       [ 7.12361848e+08,  1.12550000e+00, -9.70900000e-01, ...,
                    nan,             nan,             nan],
       [ 7.12361848e+08,  1.15780000e+00, -9.54200000e-01, ...,
                    nan,             nan,             nan],
       [ 7.12361848e+08,  1.19180000e+00, -9.27200000e-01, ...,
                    nan,             nan,             nan]])

In [102]:
def ecef2geodetic_v2(
    np_array,
    semimajor_axis: float = semimajor_axis,
    semiminor_axis: float = semiminor_axis,
    eps=finfo(np.float32).eps,
) -> tuple:
    """
    convert ECEF (meters) to geodetic coordinates

    rewrite of pymap3d.ecef2geodetic to use numba for speedup...
    not as robust
    """
    
    x, y, z = np_array[:,4], np_array[:,5], np_array[:,6]

    r = sqrt(x**2 + y**2 + z**2)

    E = sqrt(semimajor_axis**2 - semiminor_axis**2)

    # eqn. 4a
    u = sqrt(0.5 * (r**2 - E**2) + 0.5 * hypot(r**2 - E**2, 2 * E * z))

    hxy = hypot(x, y)

    huE = hypot(u, E)

    # eqn. 4b
    Beta = empty_like(r)
    Beta = atan(huE / u * z / hxy)
    # eqn. 13
    Beta += ((semiminor_axis * u - semimajor_axis * huE + E**2) * sin(Beta)) / (
        semimajor_axis * huE * 1 / cos(Beta) - E**2 * cos(Beta)
    )

    # eqn. 4c
    # %% final output
    lat = atan(semimajor_axis / semiminor_axis * tan(Beta))

    # # patch latitude for float32 precision loss
    lim_pi2 = pi / 2 - eps
    lat = where(Beta >= lim_pi2, pi / 2, lat)
    lat = where(Beta <= -lim_pi2, -pi / 2, lat)

    lon = atan2(y, x)

    # eqn. 7
    cosBeta = cos(Beta)

    # patch altitude for float32 precision loss
    cosBeta = where(Beta >= lim_pi2, 0, cosBeta)
    cosBeta = where(Beta <= -lim_pi2, 0, cosBeta)

    alt = hypot(z - semiminor_axis * sin(Beta), hxy - semimajor_axis * cosBeta)

    lat = degrees(lat)
    lon = degrees(lon)
    
    np_array[:, 19] = lat
    np_array[:, 20] = lon
    np_array[:, 21] = alt

    return np_array


In [103]:
%%time

records_copy_v2 = ecef2geodetic_v2(
    records_copy
)
records_copy_v2

CPU times: user 507 ms, sys: 67.7 ms, total: 575 ms
Wall time: 575 ms


array([[ 7.12293533e+08,  3.41000000e-02, -4.85000000e-01, ...,
         4.53022793e+01, -1.24965962e+02,  2.74717960e+01],
       [ 7.12293533e+08,  4.38000000e-02, -4.74000000e-01, ...,
                    nan,             nan,             nan],
       [ 7.12293533e+08,  5.76000000e-02, -4.70200000e-01, ...,
                    nan,             nan,             nan],
       ...,
       [ 7.12361848e+08,  1.12550000e+00, -9.70900000e-01, ...,
                    nan,             nan,             nan],
       [ 7.12361848e+08,  1.15780000e+00, -9.54200000e-01, ...,
                    nan,             nan,             nan],
       [ 7.12361848e+08,  1.19180000e+00, -9.27200000e-01, ...,
                    nan,             nan,             nan]])

In [104]:
np.array_equal(records_copy_v1, records_copy_v2, equal_nan=True)

True

In [ ]:
x

In [ ]:
P

In [ ]:
K

In [ ]:
Pp

In [ ]:
smoothed_results = pd.DataFrame(
    x.reshape(x.shape[0], -1), columns=["x", "y", "z", "vx", "vy", "vz"]
)
lats, lons, alts = pymap3d.ecef2geodetic(
    smoothed_results.x.values, smoothed_results.y.values, smoothed_results.z.values
)
smoothed_results["lat"] = lats
smoothed_results["lon"] = lons
smoothed_results["hae"] = alts
smoothed_results["dts"] = merged_df[0 : len(smoothed_results)].dts

In [ ]:
smoothed_results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
smoothed_results.plot("lon", "lat", ax=ax, c="green", label="smoothed")
compare = merged_df[0 : len(smoothed_results)][
    ~merged_df[0 : len(smoothed_results)].z.isnull()
]
compare.reset_index(drop=True, inplace=True)
lats, lons, alts = pymap3d.ecef2geodetic(
    compare.x.values, compare.y.values, compare.z.values
)
compare["lat"] = lats
compare["lon"] = lons
compare["hae"] = alts
compare.plot("lon", "lat", ax=ax, c="blue", label="GNSS")
plt.show()

In [ ]:
fig, ax = plt.subplots()
smoothed_results.plot("dts", "hae", ax=ax, c="green", label="smoothed")
compare = merged_df[0 : len(smoothed_results)][
    ~merged_df[0 : len(smoothed_results)].z.isnull()
]
compare.reset_index(drop=True, inplace=True)
lats, lons, alts = pymap3d.ecef2geodetic(
    compare.x.values, compare.y.values, compare.z.values
)
compare["lat"] = lats
compare["lon"] = lons
compare["hae"] = alts
compare.plot("dts", "hae", ax=ax, c="blue", label="GNSS")
plt.show()